In [1]:
import snowflake.connector as sf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM


In [2]:
# Define the S3 bucket and Snowflake parameters
snowflake_account = 'BF48652.ap-south-1.aws'
snowflake_user = 'FYP2'
snowflake_password = '!Q2w3e4r5t'
snowflake_database = 'COAPDATA'
snowflake_schema = 'PUBLIC'
snowflake_warehouse = 'COMPUTE_WH'
snowflake_table = 'mlview'


In [3]:
# Define the model parameters
n_input = 10
n_features = 1
model_file_name = 'model.h5'


In [4]:
# load the initial LSTM model
model = load_model('model.h5')

In [5]:
conn = sf.connect(user=snowflake_user, password=snowflake_password, account=snowflake_account)
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM {snowflake_database}.{snowflake_schema}.{snowflake_table} ORDER BY Date DESC")
data = cursor.fetchall()
cursor.close()
conn.close()

In [6]:
df = pd.DataFrame(data, columns=['Date', 'Production'])
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)
    

In [7]:
df

,Production
Date,
2021-06-18,1456
2021-06-19,1632
2021-06-20,1211
2021-06-21,1943
2021-06-22,1367
2021-06-23,1578
2021-06-24,1784
2021-06-25,1125
2021-06-26,1249


In [8]:
 # Scale the data using the same scaler used for training
scaler = MinMaxScaler()
scaler.fit(df)
scaled_data = scaler.transform(df)

In [9]:
scaled_data

array([[0.38399072],
       [0.58816705],
       [0.09976798],
       [0.94895592],
       [0.28074246],
       [0.52552204],
       [0.76450116],
       [0.        ],
       [0.14385151],
       [1.        ],
       [0.76450116],
       [0.58816705],
       [0.        ],
       [0.14385151],
       [1.        ],
       [0.38399072],
       [0.52552204],
       [0.28074246],
       [0.94895592],
       [0.09976798]])

In [10]:
scaled_data.shape

(20, 1)

In [11]:
 # Update the model with the new data
generator = TimeseriesGenerator(scaled_data, scaled_data, length=n_input, batch_size=1)


In [12]:
last_scaled_batch = scaled_data[-10:]

In [13]:
last_scaled_batch = last_scaled_batch.reshape((1, n_input, n_features))

In [14]:
model.fit(generator, epochs=1, verbose=0)

In [15]:
prediction = model.predict(last_scaled_batch)
true_prediction = scaler.inverse_transform(prediction)[0][0]

In [16]:
true_prediction

2004.3899

In [18]:
snowflake_conn = sf.connect(
        user='FYP2',
        password='!Q2w3e4r5t',
        account='BF48652.ap-south-1.aws',
        warehouse='COMPUTE_WH',
        database='COAPDATA',
        schema='PUBLIC'
)


In [19]:
cursor = snowflake_conn.cursor()
cursor.execute(f"INSERT INTO predict_table (date, prediction) VALUES (DATEADD(DAY, 1, (SELECT MAX(date) FROM mlview)), {true_prediction})")
cursor.close()
snowflake_conn.close()